In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd

import random

import gc

%matplotlib inline

# Load Data

In [2]:
train_data = pd.read_csv('train.csv')

In [98]:
target = train_data['target']
context = train_data['context']
label = train_data['label']

In [39]:
label.shape

(64483270,)

In [6]:
anchor_list = pd.read_csv('../viewData/anchor_list.csv')

In [7]:
anchor_list[:10]

,live_uid,count
0,730183576,558593
1,16673072,123029
2,4994793,95888
3,21678553,94230
4,735415548,90641
5,671116,87475
6,27009880,85699
7,124929108,82722
8,106059379,82672
9,723809182,82649


In [8]:
anchor_list = anchor_list.reset_index()
anchor_list['index'] = anchor_list['index'] + 1

In [9]:
frequent_anchor = anchor_list[anchor_list['count']>=10]
index2uid = dict([(i, uid) for i, uid in zip(frequent_anchor['index'], frequent_anchor['live_uid'])])
uid2index = dict([(uid, i) for uid, i in zip(frequent_anchor['live_uid'], frequent_anchor['index'])])

# Construct Model

In [14]:
def item2vecModel(vocab_size, embedding_dim, optimizer='rmsprop'):
    target = keras.layers.Input(shape=(1,), name='target')
    context = keras.layers.Input(shape=(1,), name='context')
    shared_embedding = keras.layers.Embedding(vocab_size, embedding_dim, input_length=1, name='shared_embedding')
    embedding_target = shared_embedding(target)
    embedding_context = shared_embedding(context)
    merged_vector = keras.layers.dot([embedding_target, embedding_context], axes=-1)
    reshaped_vector = keras.layers.Reshape((1,), input_shape=(1,1))(merged_vector)
    prediction = keras.layers.Dense(1, input_shape=(1,), activation='sigmoid')(reshaped_vector)
    model = keras.models.Model(inputs=[target, context], outputs=prediction)
    model.compile(optimizer=optimizer, loss='binary_crossentropy')
    return model

In [47]:
V = len(frequent_anchor) + 1

In [48]:
D = 100

In [49]:
model = item2vecModel(V, D, optimizer='adam')

In [31]:
from datetime import datetime

In [99]:
target = target.values.T.reshape(len(target),1)
context = context.values.T.reshape(len(target),1)
label = label.values.T.reshape(len(target),1)

In [116]:
model.fit([target, context], label, epochs=1)

Train on 64483270 samples
  220768/64483270 [..............................] - ETA: 25:43:20 - loss: 0.4535

KeyboardInterrupt: 